In [23]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic-dataset/Titanic-Dataset.csv


In [24]:
# Importar bibliotecas necessárias
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import joblib
import pickle
import warnings
warnings.filterwarnings('ignore')

In [25]:
# ==========================
# 1. CARREGAR DADOS
# ==========================
print("=" * 60)
print("ANÁLISE DO DATASET TITANIC - REGRESSÃO LINEAR")
print("=" * 60)

# Carregar o dataset
df = pd.read_csv('/kaggle/input/titanic-dataset/Titanic-Dataset.csv')

print("\n📊 Primeiras linhas do dataset:")
print(df.head())

print(f"\n📏 Dimensões: {df.shape[0]} linhas x {df.shape[1]} colunas")


ANÁLISE DO DATASET TITANIC - REGRESSÃO LINEAR

📊 Primeiras linhas do dataset:
   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53

In [26]:
# ==========================
# 2. ANÁLISE EXPLORATÓRIA
# ==========================
print("\n" + "=" * 60)
print("ANÁLISE EXPLORATÓRIA DOS DADOS")
print("=" * 60)

print("\n🔍 Informações do dataset:")
print(df.info())

print("\n📈 Estatísticas descritivas:")
print(df.describe())

print("\n❓ Valores nulos:")
print(df.isnull().sum())

print("\n📊 Estatísticas da variável alvo (Fare):")
print(f"Média: ${df['Fare'].mean():.2f}")
print(f"Mediana: ${df['Fare'].median():.2f}")
print(f"Desvio padrão: ${df['Fare'].std():.2f}")
print(f"Mínimo: ${df['Fare'].min():.2f}")
print(f"Máximo: ${df['Fare'].max():.2f}")

print("\n📊 Tarifa média por Classe:")
print(df.groupby('Pclass')['Fare'].mean().sort_values(ascending=False))

print("\n📊 Tarifa média por Sexo:")
print(df.groupby('Sex')['Fare'].mean())



ANÁLISE EXPLORATÓRIA DOS DADOS

🔍 Informações do dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None

📈 Estatísticas descritivas:
       PassengerId    Survived      Pclass         Age       SibSp  \
count   891.000000  891.000000  891.000000  714.000000  891.000000   
mean    446

In [27]:
# ==========================
# 3. PRÉ-PROCESSAMENTO
# ==========================
print("\n" + "=" * 60)
print("PRÉ-PROCESSAMENTO DOS DADOS")
print("=" * 60)

# Criar cópia para processamento
df_processed = df.copy()

# Remover linhas onde Fare é nulo (variável alvo)
print(f"\n🔧 Removendo linhas com Fare nulo...")
print(f"Linhas antes: {len(df_processed)}")
df_processed = df_processed[df_processed['Fare'].notna()]
print(f"Linhas depois: {len(df_processed)}")

# Preencher valores nulos nas features
print("\n🔧 Tratando valores nulos nas features...")
df_processed['Age'].fillna(df_processed['Age'].median(), inplace=True)
df_processed['Embarked'].fillna(df_processed['Embarked'].mode()[0], inplace=True)

print("Valores nulos após tratamento:")
print(df_processed.isnull().sum())

# Remover coluna Cabin (muitos nulos)
if 'Cabin' in df_processed.columns:
    df_processed.drop('Cabin', axis=1, inplace=True)

# Codificar variáveis categóricas
print("\n🔧 Codificando variáveis categóricas...")
le_sex = LabelEncoder()
df_processed['Sex'] = le_sex.fit_transform(df_processed['Sex'])

if 'Embarked' in df_processed.columns:
    le_embarked = LabelEncoder()
    df_processed['Embarked'] = le_embarked.fit_transform(df_processed['Embarked'])

# Criar novas features
print("🔧 Criando novas features...")
df_processed['FamilySize'] = df_processed['SibSp'] + df_processed['Parch'] + 1
df_processed['IsAlone'] = (df_processed['FamilySize'] == 1).astype(int)

print(f"FamilySize - Min: {df_processed['FamilySize'].min()}, Max: {df_processed['FamilySize'].max()}")
print(f"IsAlone - Distribuição:\n{df_processed['IsAlone'].value_counts()}")

# Selecionar features para o modelo
features_to_use = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Survived', 'FamilySize', 'IsAlone']
if 'Embarked' in df_processed.columns:
    features_to_use.append('Embarked')

X = df_processed[features_to_use]
y = df_processed['Fare']

print(f"\n✅ Features selecionadas: {features_to_use}")
print(f"✅ Variável alvo: Fare (Tarifa)")
print(f"✅ Shape dos dados: X={X.shape}, y={y.shape}")



PRÉ-PROCESSAMENTO DOS DADOS

🔧 Removendo linhas com Fare nulo...
Linhas antes: 891
Linhas depois: 891

🔧 Tratando valores nulos nas features...
Valores nulos após tratamento:
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
dtype: int64

🔧 Codificando variáveis categóricas...
🔧 Criando novas features...
FamilySize - Min: 1, Max: 11
IsAlone - Distribuição:
IsAlone
1    537
0    354
Name: count, dtype: int64

✅ Features selecionadas: ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Survived', 'FamilySize', 'IsAlone', 'Embarked']
✅ Variável alvo: Fare (Tarifa)
✅ Shape dos dados: X=(891, 9), y=(891,)


In [28]:
# ==========================
# 4. DIVISÃO TREINO/TESTE
# ==========================
print("\n" + "=" * 60)
print("DIVISÃO DOS DADOS")
print("=" * 60)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"\n📊 Tamanho do conjunto de treino: {X_train.shape[0]} amostras")
print(f"📊 Tamanho do conjunto de teste: {X_test.shape[0]} amostras")
print(f"\nEstatísticas da tarifa no treino:")
print(f"  Média: ${y_train.mean():.2f}")
print(f"  Mediana: ${y_train.median():.2f}")
print(f"\nEstatísticas da tarifa no teste:")
print(f"  Média: ${y_test.mean():.2f}")
print(f"  Mediana: ${y_test.median():.2f}")

# Normalizar os dados
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("\n✅ Dados normalizados com StandardScaler")


DIVISÃO DOS DADOS

📊 Tamanho do conjunto de treino: 712 amostras
📊 Tamanho do conjunto de teste: 179 amostras

Estatísticas da tarifa no treino:
  Média: $32.59
  Mediana: $14.45

Estatísticas da tarifa no teste:
  Média: $30.68
  Mediana: $14.50

✅ Dados normalizados com StandardScaler


In [29]:
# ==========================
# 5. TREINAMENTO DOS MODELOS
# ==========================
print("\n" + "=" * 60)
print("TREINAMENTO DOS MODELOS DE REGRESSÃO")
print("=" * 60)

# Dicionário de modelos
models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(alpha=1.0, random_state=42),
    'Lasso Regression': Lasso(alpha=1.0, random_state=42)
}

results = {}

for name, model in models.items():
    print(f"\n🚀 Treinando {name}...")
    
    # Treinar modelo
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    
    # Cross-validation (R² score negativo no sklearn)
    cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='r2')
    
    # Calcular métricas
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results[name] = {
        'model': model,
        'predictions': y_pred,
        'mse': mse,
        'rmse': rmse,
        'mae': mae,
        'r2': r2,
        'cv_mean': cv_scores.mean(),
        'cv_std': cv_scores.std()
    }
    
    print(f"✅ MSE (Mean Squared Error): {mse:.4f}")
    print(f"✅ RMSE (Root Mean Squared Error): ${rmse:.2f}")
    print(f"✅ MAE (Mean Absolute Error): ${mae:.2f}")
    print(f"✅ R² Score: {r2:.4f}")
    print(f"📊 Cross-validation R² (5-fold): {cv_scores.mean():.4f} (+/- {cv_scores.std():.4f})")



TREINAMENTO DOS MODELOS DE REGRESSÃO

🚀 Treinando Linear Regression...
✅ MSE (Mean Squared Error): 935.2756
✅ RMSE (Root Mean Squared Error): $30.58
✅ MAE (Mean Absolute Error): $20.89
✅ R² Score: 0.3956
📊 Cross-validation R² (5-fold): 0.3756 (+/- 0.1094)

🚀 Treinando Ridge Regression...
✅ MSE (Mean Squared Error): 934.4372
✅ RMSE (Root Mean Squared Error): $30.57
✅ MAE (Mean Absolute Error): $20.87
✅ R² Score: 0.3961
📊 Cross-validation R² (5-fold): 0.3757 (+/- 0.1091)

🚀 Treinando Lasso Regression...
✅ MSE (Mean Squared Error): 909.9775
✅ RMSE (Root Mean Squared Error): $30.17
✅ MAE (Mean Absolute Error): $19.89
✅ R² Score: 0.4119
📊 Cross-validation R² (5-fold): 0.3815 (+/- 0.1020)


In [30]:
# ==========================
# 6. AVALIAÇÃO DOS MODELOS
# ==========================
print("\n" + "=" * 60)
print("AVALIAÇÃO E COMPARAÇÃO DOS MODELOS")
print("=" * 60)

# Comparar modelos
print("\n📊 Resumo dos Resultados:")
print("-" * 80)
print(f"{'Modelo':<20} {'RMSE':<12} {'MAE':<12} {'R²':<12} {'CV R²':<15}")
print("-" * 80)
for name, result in sorted(results.items(), key=lambda x: x[1]['r2'], reverse=True):
    print(f"{name:<20} ${result['rmse']:<11.2f} ${result['mae']:<11.2f} {result['r2']:<12.4f} {result['cv_mean']:.4f} (+/- {result['cv_std']:.4f})")

# Melhor modelo
best_model_name = max(results, key=lambda x: results[x]['r2'])
best_model = results[best_model_name]['model']
best_predictions = results[best_model_name]['predictions']

print(f"\n🏆 Melhor modelo: {best_model_name}")
print(f"🎯 R² Score: {results[best_model_name]['r2']:.4f}")
print(f"🎯 RMSE: ${results[best_model_name]['rmse']:.2f}")
print(f"🎯 MAE: ${results[best_model_name]['mae']:.2f}")

# Análise de resíduos
residuals = y_test - best_predictions
print(f"\n📊 Análise de Resíduos ({best_model_name}):")
print("-" * 60)
print(f"Média dos resíduos: ${residuals.mean():.2f}")
print(f"Desvio padrão dos resíduos: ${residuals.std():.2f}")
print(f"Resíduo mínimo: ${residuals.min():.2f}")
print(f"Resíduo máximo: ${residuals.max():.2f}")

# Coeficientes do modelo
print(f"\n📊 Coeficientes do Modelo ({best_model_name}):")
print("-" * 60)
coef_df = pd.DataFrame({
    'Feature': features_to_use,
    'Coeficiente': best_model.coef_
}).sort_values('Coeficiente', key=abs, ascending=False)
print(coef_df.to_string(index=False))
print(f"\nIntercepto: ${best_model.intercept_:.2f}")



AVALIAÇÃO E COMPARAÇÃO DOS MODELOS

📊 Resumo dos Resultados:
--------------------------------------------------------------------------------
Modelo               RMSE         MAE          R²           CV R²          
--------------------------------------------------------------------------------
Lasso Regression     $30.17       $19.89       0.4119       0.3815 (+/- 0.1020)
Ridge Regression     $30.57       $20.87       0.3961       0.3757 (+/- 0.1091)
Linear Regression    $30.58       $20.89       0.3956       0.3756 (+/- 0.1094)

🏆 Melhor modelo: Lasso Regression
🎯 R² Score: 0.4119
🎯 RMSE: $30.17
🎯 MAE: $19.89

📊 Análise de Resíduos (Lasso Regression):
------------------------------------------------------------
Média dos resíduos: $-6.04
Desvio padrão dos resíduos: $29.64
Resíduo mínimo: $-63.04
Resíduo máximo: $158.74

📊 Coeficientes do Modelo (Lasso Regression):
------------------------------------------------------------
   Feature  Coeficiente
    Pclass   -26.569299
FamilySi

In [31]:
# ==========================
# 7. PREDIÇÕES DE EXEMPLO
# ==========================
print("\n" + "=" * 60)
print("EXEMPLOS DE PREDIÇÃO")
print("=" * 60)

# Mostrar alguns exemplos de predição
examples = X_test.head(10)
examples_scaled = scaler.transform(examples)
predictions = best_model.predict(examples_scaled)

print("\nPrimeiros 10 passageiros do conjunto de teste:")
print("-" * 80)
print(f"{'Real':<12} {'Predito':<12} {'Erro':<12} {'Classe':<8} {'Sexo':<8} {'Idade':<8}")
print("-" * 80)
for i in range(len(examples)):
    real = y_test.iloc[i]
    pred = predictions[i]
    erro = abs(real - pred)
    print(f"${real:<11.2f} ${pred:<11.2f} ${erro:<11.2f} {examples.iloc[i]['Pclass']:<8} {examples.iloc[i]['Sex']:<8} {examples.iloc[i]['Age']:<8.0f}")



EXEMPLOS DE PREDIÇÃO

Primeiros 10 passageiros do conjunto de teste:
--------------------------------------------------------------------------------
Real         Predito      Erro         Classe   Sexo     Idade   
--------------------------------------------------------------------------------
$15.25       $36.07       $20.82       3.0      1.0      28      
$10.50       $30.79       $20.29       2.0      1.0      31      
$7.92        $-1.45       $9.38        3.0      1.0      20      
$33.00       $45.17       $12.17       2.0      0.0      6       
$11.24       $26.24       $15.00       3.0      0.0      14      
$78.85       $67.09       $11.76       1.0      0.0      26      
$7.75        $11.65       $3.90        3.0      0.0      28      
$18.00       $9.65        $8.35        3.0      1.0      16      
$7.75        $11.65       $3.90        3.0      0.0      16      
$26.28       $87.74       $61.46       1.0      0.0      19      


In [32]:
 ==========================
# 8. EXPORTAR O MODELO
# ==========================
print("\n" + "=" * 60)
print("EXPORTAR MODELO E SCALER")
print("=" * 60)

import joblib

# Exportar o melhor modelo
joblib.dump(best_model, 'modelo_titanic.pkl')
print(f"✅ Modelo exportado: modelo_titanic.pkl")

# Exportar o scaler
joblib.dump(scaler, 'scaler_titanic.pkl')
print(f"✅ Scaler exportado: scaler_titanic.pkl")

# Exportar informações adicionais
model_info = {
    'model_name': best_model_name,
    'features': features_to_use,
    'r2_score': results[best_model_name]['r2'],
    'rmse': results[best_model_name]['rmse'],
    'mae': results[best_model_name]['mae']
}
joblib.dump(model_info, 'model_info.pkl')
print(f"✅ Informações do modelo exportadas: model_info.pkl")

print("\n💡 Para carregar o modelo depois:")
print("   modelo = joblib.load('modelo_titanic.pkl')")
print("   scaler = joblib.load('scaler_titanic.pkl')")
print("   info = joblib.load('model_info.pkl')")


SyntaxError: invalid syntax (2855937341.py, line 1)

In [ ]:
# ==========================
# 9. CONCLUSÕES
# ==========================
print("\n" + "=" * 60)
print("CONCLUSÕES")
print("=" * 60)
print(f"""
✅ Modelo de regressão treinado com sucesso!
🎯 Melhor modelo: {best_model_name}
📊 R² Score: {results[best_model_name]['r2']:.4f}
📊 RMSE: ${results[best_model_name]['rmse']:.2f}
📊 MAE: ${results[best_model_name]['mae']:.2f}
🔍 Total de passageiros analisados: {len(df_processed)}
📈 Cross-validation R²: {results[best_model_name]['cv_mean']:.4f}

💡 O modelo pode prever a tarifa (Fare) paga por um passageiro
   com base em características como classe, sexo, idade, família
   e status de sobrevivência.
   
📌 Interpretação do R²: {results[best_model_name]['r2']*100:.2f}% da variabilidade
   da tarifa é explicada pelo modelo.
""")

print("=" * 60)
print("ANÁLISE CONCLUÍDA!")
print("=" * 60)